# Visualização e manipulação de imagens FITS

## Autores
Lia Corrales, Kris Stern, Stephanie T. Douglas, Kelle Cruz

## Tradução
Filipe Góis

## Objetivos
* Abrir arquivos FITS e carregar dados da imagem
2. Fazer um histograma 2D com dados da imagem
3. Juntar várias imagens em uma única imagem
4. Escreve dados da imagem em um arquivo FITS

## Palavras-chave
FITS, entrada/saída de arquivo, manipulação de imagem, numpy, matplotlib, histograma, barra de cor

## Resumo

Este tutorial mostra o uso do astropy.utils.data para baixar um arquivo de dados, então usa astropy.io.fits para abrir o arquivo, e, por fim, usa matplotlib para vizualizar a imagem com diferentes escalas de cores, stretches e para fazer histogramas. Neste tutorial, incluímos, também, uma demonstração de uma junção simples de imagens

Geralmente, a informação da imagem está localizada no bloco <code>PRIMARY</code>. Os blocos são numerado e podem ser acessados indexando <code>hud_list</code>.

In [ ]:
import numpy as np

# Set up matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

from astropy.io import fits

Baixe os arquivos FITS para este tutorial

In [ ]:
from astropy.utils.data import download_file
image_file = download_file('http://data.astropy.org/tutorials/FITS-images/HorseHead.fits', cache=True )

## Abrindo arquivos FITS e carregando os dados da imagem

Abramos o arquivo FITS para descobrir o que ele contém.

In [ ]:
hdu_list = fits.open(image_file)
hdu_list.info()

In [ ]:
image_data = hdu_list[0].data

Agora, nosso dado está armazenado em um *array* Numpy bidimensional. Mas como sabemos as dimensões da imagem? Podemos analisar o `shape` do *array*.

In [ ]:
print(type(image_data))
print(image_data.shape)

Ótimo! A esta altura, podemos fechar o arquivo FITS, porque já armazenamos tudo que queríamos para uma variável.

In [ ]:
hdu_list.close()

### ATALHO

Se você não precisa examinar o cabeçalho do arquivo FITS, você pode chamar a função `fits.getdata` para pular os passos anteriores.

In [ ]:
image_data = fits.getdata(image_file)
print(type(image_data))
print(image_data.shape)

## Vendo o dado da imagem e obtendo estatísticas básicas

In [ ]:
plt.imshow(image_data, cmap='gray')
plt.colorbar()

# Para ver mais mapas de cores
# http://wiki.scipy.org/Cookbook/Matplotlib/Show_colormaps

Obtenhamos algumas estatísticas básicas de nossa imagem:

In [ ]:
print('Min:', np.min(image_data))
print('Max:', np.max(image_data))
print('Mean:', np.mean(image_data))
print('Stdev:', np.std(image_data))

### Plotando um histograma

Para fazer um histograma com `matplotlib.pyplot.hist()`, precisamos transferir o dado de um *array* bidimensional para algo unidimensional.

Neste caso, usemos a função `ndarray.flatten()` para transformar um *array* Numpy unidimensional. 

In [ ]:
print(type(image_data.flatten()))

In [ ]:
histogram = plt.hist(image_data.flatten(), bins='auto')

### Mostando a imagem em escala logarítmica

E se quisermos usar uma escala logarítmica de cores? Para fazer isso, precisamos carregar o objeto `LogNorm` da biblioteca `matplotlib`.

In [ ]:
from matplotlib.colors import LogNorm

In [ ]:
plt.imshow(image_data, cmap='gray', norm=LogNorm())

# Escolhi as marcas de escala baseado no histograma acima
# I chose the tick marks based on the histogram above
cbar = plt.colorbar(ticks=[5.e3,1.e4,2.e4])
cbar.ax.set_yticklabels(['5,000','10,000','20,000'])

## Matemática Básica com a imagem: junção de imagens

Você também pode fazer cálculos com o dado da imagem com qualquer outro *array*  Numpy. Neste exemplo em particular, juntaremos várias imagens de M13 tiradas com um telescópio de ~10".

Comecemos abrindo uma série de aruivo FITS e armazenando o dado em uma lista, a qual chamamos de `image_concat`.

In [ ]:
base_url = 'http://data.astropy.org/tutorials/FITS-images/M13_blue_{0:04d}.fits'

image_list = [download_file(base_url.format(n), cache=True) 
              for n in range(1, 5+1)]
image_concat = [fits.getdata(image) for image in image_list]

Agora, juntaremos as imagens chamando a lista concatenada.

In [ ]:
# O caminho mais longo
final_image = np.zeros(shape=image_concat[0].shape)

for image in image_concat:
    final_image += image

# O caminho mais curto
# final_image = np.sum(image_concat, axis=0)

Vamos mostrar a imagem, mas precisamos decidir qual o melhor trecho. Para isso, plotemos um histograma do dado. 

In [ ]:
image_hist = plt.hist(final_image.flatten(), bins='auto')

Usaremos as palavras-chave `vmin` and `vmax` para limitar a escala de cores para `imshow`.

In [ ]:
plt.imshow(final_image, cmap='gray', vmin=2E3, vmax=3E3)
plt.colorbar()

## Escrevendo o dado da imagem em um arquivo FITS

Podemos fazer isso com o método `writeto()`.

**AVISO:** você receberá uma mensagem de erro se o arquivo que você está tentando escrever já existe. Por isso, colocamos `overwrite=True`.

In [ ]:
outfile = 'stacked_M13_blue.fits'

hdu = fits.PrimaryHDU(final_image)
hdu.writeto(outfile, overwrite=True)

## Exercícios

Determine a média, mediana e o desvio-padrão da parte da imagem juntada de M13 onde *não* há luz de M13. Use estas estatísticas com uma soma sobre a parte da imagem que inclui M13 para estimar a luz total nesta imagem de M13. 

Mostre a image da Nebulosa Cabeça de Cavalo, mas em unidade de *brilho superficial* (magnitudes por segundo de arco ao quadrado). 
(Dica: o tamanho *físico* da imagem é 15x15 minutos de arco.)

Agora, salve a imagem que você acabou de criar, preservando o cabeçalho da imagem original, mas adicione a chave 'UNITS' com o valor 'mag per sq arcsec'. 
(Dica: leia a documentação de [astropy.io.fits](http://docs.astropy.org/en/stable/io/fits/index.html) se você não sabe como incluir o cabeçalho e o dado.)